In [ ]:
# %%
# for gridsearch

#!pip install pyyaml==5.4.1

# %%
from IPython import get_ipython  # type: ignore

# %% 
get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')


# %%

from dotmap import DotMap
import json
import os

import numpy as np

# Import the PyTorch modules
import torch  # type: ignore
from torch import nn  # type: ignore
from torch.optim import SGD, Adam, RMSprop  # type: ignore

# Import Tensorflow writer
from torch.utils.tensorboard import SummaryWriter  # type: ignore

# Import modules from XTransformers
from x_transformers.x_transformers import AttentionLayers, Encoder, ContinuousTransformerWrapper


# Import the giotto-deep modules
from gdeep.data import OrbitsGenerator, DataLoaderKwargs
from gdeep.topology_layers import SetTransformer, PersFormer
#from gdeep.topology_layers import AttentionPooling
from gdeep.topology_layers import ISAB, PMA, SAB
from gdeep.pipeline import Pipeline
from gdeep.search import Gridsearch
import json
#from gdeep.search import Gridsearch

from optuna.pruners import MedianPruner, NopPruner

# %%

#Configs
config_data = DotMap({
    'batch_size_train': 32,
    'num_orbits_per_class': 2_000,
    'validation_percentage': 0.0,
    'test_percentage': 0.0,
    'num_jobs': 2,
    'dynamical_system': 'classical_convention',
    'homology_dimensions': (0, 1),
    'dtype': 'float32',
    'arbitrary_precision': False
})


config_model = DotMap({
    'implementation': 'Old_SetTransformer', # SetTransformer, PersFormer,
    # PytorchTransformer, DeepSet, X-Transformer
    'dim_input': 4,
    'num_outputs': 1,  # for classification tasks this should be 1
    'num_classes': 5,  # number of classes
    'dim_hidden': 64,
    'num_heads': 4,
    'num_induced_points': 64,
    'layer_norm': False,  # use layer norm
    'pre_layer_norm': False,
    'num_layers_encoder': 2,
    'num_layers_decoder': 1,
    'attention_type': "self_attention",
    'activation': "nn.ReLU()",
    'dropout_enc': 0.0,
    'dropout_dec': 0.0,
    'optimizer': torch.optim.Adam,
    'learning_rate': 1e-4,
    'num_epochs': 500,
    'pooling_type': "attention",
    'weight_decay': 0.0,
    'n_accumulated_grads': 0,
    'bias_attention': True
})



# %%



# Define the data loader


dataloaders_dicts = DataLoaderKwargs(train_kwargs = {"batch_size":
                                                        config_data.batch_size_train,},
                                     val_kwargs = {"batch_size": 4},
                                     test_kwargs = {"batch_size": 3})

og = OrbitsGenerator(num_orbits_per_class=config_data.num_orbits_per_class,
                     homology_dimensions = config_data.homology_dimensions,
                     validation_percentage=config_data.validation_percentage,
                     test_percentage=config_data.test_percentage,
                     n_jobs=config_data.num_jobs,
                     dynamical_system = config_data.dynamical_system,
                     dtype=config_data.dtype,
                     arbitrary_precision=config_data.arbitrary_precision,
                     )

if config_data.arbitrary_precision:
    orbits = np.load(os.path.join('data', 'orbit5k_arbitrary_precision.npy'))
    og.orbits_from_array(orbits)

if config_data.dim_input == 2:
    dl_train, _, _ = og.get_dataloader_orbits(dataloaders_dicts)
else:
    dl_train, _, _ = og.get_dataloader_persistence_diagrams(dataloaders_dicts)



# Define the model
if config_model.implementation == 'SetTransformer':
    model = SetTransformer(
            dim_input=config_model.dim_input,
            num_outputs=1,  # for classification tasks this should be 1
            dim_output=config_model.num_classes,  # number of classes
            dim_hidden=config_model.dim_hidden,
            num_heads=config_model.num_heads,
            num_inds=config_model.num_induced_points,
            ln=config_model.layer_norm,  # use layer norm
            n_layers_encoder=config_model.num_layers_encoder,
            n_layers_decoder=config_model.num_layers_decoder,
            attention_type=config_model.attention_type,
            dropout=config_model.dropout
    )

elif config_model.implementation == 'PersFormer':
    model = PersFormer(
            dim_input=2,
            dim_output=5,
            n_layers=5,
            hidden_size=32,
            n_heads=4,
            dropout=0.1,
            layer_norm=True,
            pre_layer_norm=False,
            activation=nn.GELU,
            attention_layer_type="self_attention")

elif config_model.implementation == 'PytorchTransformer':
    model = PytorchTransformer(
            dim_input=2,
            dim_output=5,
            hidden_size=64,
            nhead=8,
            activation='gelu',
            norm_first=True,
            num_layers=3,
            dropout=0.0,
    )
elif config_model.implementation == 'DeepSet':
    model = DeepSet(dim_input=2,
                    dim_output=config_model.num_classes,
                    dim_hidden=config_model.dim_hidden,
                    n_layers_encoder=config_model.num_layers_encoder,
                    n_layers_decoder=config_model.num_layers_decoder,
                    pool=config_model.pooling_type).double()

elif config_model.implementation == "X-Transformer":
    model = \
    nn.Sequential(
        ContinuousTransformerWrapper(
            dim_in = 2,
            use_pos_emb = True,
            max_seq_len = None,
            attn_layers = Encoder(
                dim = config_model.dim_hidden,
                depth = config_model.num_layers_encoder,
                heads = config_model.num_heads,
            ),
        ),
        AttentionPooling(hidden_dim = config_model.dim_hidden, q_length=1),
        nn.Sequential(*[nn.Sequential(nn.Linear(config_model.dim_hidden,
                            config_model.dim_hidden),
                            nn.ReLU())
                for _ in range(config_model.num_layers_decoder)]),
        nn.Linear(config_model.dim_hidden, config_model.num_classes)
    )
    
elif config_model.implementation == "Old_SetTransformer":
    # initialize SetTransformer model
    class SetTransformerOld(nn.Module):
        """ Vanilla SetTransformer from
        https://github.com/juho-lee/set_transformer/blob/master/main_pointcloud.py
        """
        def __init__(
            self,
            dim_input=4,  # dimension of input data for each element in the set
            num_outputs=1,
            dim_output=5,  # number of classes
            num_inds=32,  # number of induced points, see  Set Transformer paper
            dim_hidden=128,
            num_heads="4",
            layer_norm="False",  # use layer norm
            dropout=0.0,
            num_layer_enc=2,
            num_layer_dec=2,
            activation="nn.Relu()",
            bias_attention=True,
            attention_type="induced_attention"
        ):
            super().__init__()
            if attention_type=="induced_attention":
                self.enc = nn.Sequential(
                    ISAB(dim_input, dim_hidden, eval(num_heads), num_inds, ln=eval(layer_norm), bias_attention=bias_attention),
                    *[ISAB(dim_hidden, dim_hidden, eval(num_heads), num_inds, ln=eval(layer_norm), bias_attention=bias_attention)
                      for _ in range(num_layer_enc-1)],
                )
            else:
                self.enc = nn.Sequential(
                    SAB(dim_input, dim_hidden, eval(num_heads), ln=eval(layer_norm), bias_attention=bias_attention),
                    *[SAB(dim_hidden, dim_hidden, eval(num_heads), ln=eval(layer_norm), bias_attention=bias_attention)
                      for _ in range(num_layer_enc-1)],
                )
            self.dec = nn.Sequential(
                nn.Dropout(dropout),
                PMA(dim_hidden, eval(num_heads), num_outputs, ln=eval(layer_norm), bias_attention=bias_attention),
                nn.Dropout(dropout),
                *[nn.Sequential(nn.Linear(dim_hidden, dim_hidden),
                                nn.ReLU(),
                                nn.Dropout(dropout)) for _ in range(num_layer_dec-1)],
                nn.Linear(dim_hidden, dim_output),
            )

        def forward(self, input):
            return self.dec(self.enc(input)).squeeze(dim=1)



    model = SetTransformerOld(dim_input=4, dim_output=5,
                           num_inds=config_model.num_induced_points,
                           dim_hidden=config_model.dim_hidden,
                           num_heads=str(config_model.num_heads),
                           layer_norm=str(config_model.layer_norm),  # use layer norm
                           dropout=config_model.dropout_dec,
                           num_layer_enc=config_model.num_layers_encoder,
                           num_layer_dec=config_model.num_layers_decoder,
                           activation=config_model.activation,
                           bias_attention=config_model.bias_attention,
                           attention_type=config_model.attention_type)
else:
    raise Exception("Unknown Implementation")
# %%

if config_data.dtype == "float64":
    print("Use float64 model")
    model = model.double()
else:
    print("use float32 model")
    print(config_model)
    print(config_data)
    print(model)

# %%
# Do training and validation

# initialise loss
loss_fn = nn.CrossEntropyLoss()

# Initialize the Tensorflow writer
#writer = SummaryWriter(comment=json.dumps(config_model.toDict())\
#                                + json.dumps(config_data.toDict()))
writer = SummaryWriter(comment=config_model.implementation)

# initialise pipeline class
pipe = Pipeline(model, [dl_train, None], loss_fn, writer)
# %%


# train the model
pipe.train(config_model.optimizer,
           config_model.num_epochs,
           cross_validation=False,
           optimizers_param={"lr": config_model.learning_rate,
                             "weight_decay": config_model.weight_decay},
           n_accumulated_grads=config_model.n_accumulated_grads)

# %%
# keep training
#pipe.train(Adam, 300, False, keep_training=True)

# %%
# %%
# Gridsearch

# initialise gridsearch
# pruner = NopPruner()
# search = Gridsearch(pipe, search_metric="accuracy", n_trials=50, best_not_last=True, pruner=pruner)

# dictionaries of hyperparameters
optimizers_params = {"lr": [1e-7, 1e-3, None, True]}#,
                      #"weight_decay": [0.0, 0.2] }
dataloaders_params = {"batch_size": [16, 32, 4]}
models_hyperparams = {"n_layer_enc": [2, 3],
                      "n_layer_dec": [1, 5],
                      "num_heads": ["2", "4", "8"],
                      "hidden_dim": ["16", "32", "64"],
                      "dropout": [0.0, 0.2, 0.05],
                      "layer_norm": ["True", "False"]}#,
                      #'pre_layer_norm': ["True", "False"]}

# starting the gridsearch
#search.start((Adam,), n_epochs=500, cross_validation=False,
#             optimizers_params=optimizers_params,
#             dataloaders_params=dataloaders_params,
#             models_hyperparams=models_hyperparams, lr_scheduler=None,
#             scheduler_params=None)


# %%
#print(search.best_val_acc_gs, search.best_val_loss_gs)
# %%
#df_res = search._results()
#df_res
#df_res.to_csv('set_transformer_grid_search.csv')
# %%

Using GPU!
No TPUs...
use float32 model
DotMap(implementation='Old_SetTransformer', dim_input=4, num_outputs=1, num_classes=5, dim_hidden=64, num_heads=4, num_induced_points=64, layer_norm=False, pre_layer_norm=False, num_layers_encoder=2, num_layers_decoder=1, attention_type='self_attention', activation='nn.ReLU()', dropout_enc=0.0, dropout_dec=0.0, optimizer=<class 'torch.optim.adam.Adam'>, learning_rate=0.0001, num_epochs=500, pooling_type='attention', weight_decay=0.0, n_accumulated_grads=0, bias_attention=True)
DotMap(batch_size_train=32, num_orbits_per_class=2000, validation_percentage=0.0, test_percentage=0.0, num_jobs=2, dynamical_system='classical_convention', homology_dimensions=(0, 1), dtype='float32', arbitrary_precision=False, dim_input=DotMap())
SetTransformerOld(
  (enc): Sequential(
    (0): SAB(
      (mab): MAB(
        (fc_q): Linear(in_features=4, out_features=64, bias=True)
        (fc_k): Linear(in_features=4, out_features=64, bias=True)
        (fc_v): Linear(in_

Batch training loss:  0.023878907089802757  	Batch training accuracy:  20.74004975124378  	[ 201 / 250 ]                     
Time taken for this epoch: 18.00s
No TPUs
Validation results: 
 Accuracy: 18.849206%,                 Avg loss: 1.606995 

Epoch 23
-------------------------------
No TPUs
Batch training loss:  0.023872753280905348  	Batch training accuracy:  21.937189054726367  	[ 201 / 250 ]                     
Time taken for this epoch: 18.00s
No TPUs
Validation results: 
 Accuracy: 23.412698%,                 Avg loss: 1.602069 

Epoch 24
-------------------------------
No TPUs
Batch training loss:  0.023878260631466388  	Batch training accuracy:  21.703980099502488  	[ 201 / 250 ]                     
Time taken for this epoch: 18.00s
No TPUs
Validation results: 
 Accuracy: 20.486111%,                 Avg loss: 1.599900 

Epoch 25
-------------------------------
No TPUs
Batch training loss:  0.023874928109088346  	Batch training accuracy:  23.227611940298505  	[ 201 / 250 

Validation results: 
 Accuracy: 32.291667%,                 Avg loss: 1.433653 

Epoch 50
-------------------------------
No TPUs
Batch training loss:  0.021623866475043604  	Batch training accuracy:  34.34390547263681  	[ 201 / 250 ]                     
Time taken for this epoch: 18.00s
No TPUs
Validation results: 
 Accuracy: 34.573413%,                 Avg loss: 1.413515 

Epoch 51
-------------------------------
No TPUs
Batch training loss:  0.020689794080174385  	Batch training accuracy:  33.95522388059701  	[ 201 / 250 ]                     
Time taken for this epoch: 18.00s
No TPUs
Validation results: 
 Accuracy: 34.771825%,                 Avg loss: 1.400479 

Epoch 52
-------------------------------
No TPUs
Batch training loss:  0.020522773562379144  	Batch training accuracy:  34.74813432835821  	[ 201 / 250 ]                     
Time taken for this epoch: 18.00s
No TPUs
Validation results: 
 Accuracy: 36.011905%,                 Avg loss: 1.386060 

Epoch 53
----------------

Batch training loss:  0.01963332161974551  	Batch training accuracy:  38.914800995024876  	[ 201 / 250 ]                     
Time taken for this epoch: 18.00s
No TPUs
Validation results: 
 Accuracy: 37.698413%,                 Avg loss: 1.310714 

Epoch 78
-------------------------------
No TPUs
Batch training loss:  0.01995223434410285  	Batch training accuracy:  39.381218905472636  	[ 201 / 250 ]                     
Time taken for this epoch: 18.00s
No TPUs
Validation results: 
 Accuracy: 38.392857%,                 Avg loss: 1.318306 

Epoch 79
-------------------------------
No TPUs
Batch training loss:  0.01882915710335347  	Batch training accuracy:  41.55783582089552  	[ 201 / 250 ]                     
Time taken for this epoch: 18.00s
No TPUs
Validation results: 
 Accuracy: 40.972222%,                 Avg loss: 1.238808 

Epoch 80
-------------------------------
No TPUs
Batch training loss:  0.0180469051522402  	Batch training accuracy:  44.931592039801  	[ 201 / 250 ]       

Validation results: 
 Accuracy: 77.777778%,                 Avg loss: 0.563212 

Epoch 105
-------------------------------
No TPUs
Batch training loss:  0.009467899502806403  	Batch training accuracy:  79.15111940298507  	[ 201 / 250 ]                     
Time taken for this epoch: 18.00s
No TPUs
Validation results: 
 Accuracy: 79.365079%,                 Avg loss: 0.540683 

Epoch 106
-------------------------------
No TPUs
Batch training loss:  0.00807797463972177  	Batch training accuracy:  80.03731343283582  	[ 201 / 250 ]                      
Time taken for this epoch: 18.00s
No TPUs
Validation results: 
 Accuracy: 78.273810%,                 Avg loss: 0.564624 

Epoch 107
-------------------------------
No TPUs
Batch training loss:  0.007707584853196025  	Batch training accuracy:  80.06840796019901  	[ 201 / 250 ]                     
Time taken for this epoch: 18.00s
No TPUs
Validation results: 
 Accuracy: 79.811508%,                 Avg loss: 0.525085 

Epoch 108
------------

In [15]:
! python Set_transformer_benchmark_orbit5k_persistence.py > out.txt

DotMap(_ipython_display_=DotMap(), _repr_mimebundle_=DotMap())

In [4]:
df_res = search._results()
df_res

Study statistics: 
Number of pruned trials:  0
Number of complete trials:  50
Best trial:
Metric Value for best trial:  99.10714285714286


/home/jovyan/conda-envs/giottodeep/lib/python3.8/site-packages/numpy/lib/function_base.py:2474: RuntimeWarning:

invalid value encountered in subtract



,model,dataset,optimizer,lr,batch_size,dropout,n_layer_enc,n_layer_dec,num_heads,hidden_dim,layer_norm,pre_layer_norm,loss,accuracy
0,model,dataset,Adam,0.000425,32,0.158746,4,4,4,64,True,True,inf,94.921875
1,model,dataset,Adam,0.000946,48,0.130351,3,4,8,32,True,False,inf,98.710317
2,model,dataset,Adam,0.000823,32,0.044103,5,2,8,16,True,False,inf,97.070312
3,model,dataset,Adam,0.000339,32,0.174644,3,3,2,32,False,True,inf,63.769531
4,model,dataset,Adam,0.000835,64,0.016269,2,1,2,16,True,True,inf,96.191406
5,model,dataset,Adam,0.000078,32,0.036563,2,3,4,64,True,False,inf,89.160156
6,model,dataset,Adam,0.000218,32,0.126969,5,5,8,16,True,False,inf,94.628906
7,model,dataset,Adam,0.000901,64,0.154917,2,4,8,16,False,False,inf,96.582031
8,model,dataset,Adam,0.000550,32,0.125399,2,1,2,64,True,False,inf,96.875000
9,model,dataset,Adam,0.000493,32,0.004410,3,1,2,64,False,False,inf,96.777344


In [6]:
import torch_optimizer as optim

# model = ...
optimizer = optim.Shampoo(
    m.parameters(),
    lr=1e-1,
    momentum=0.0,
    weight_decay=0.0,
    epsilon=1e-4,
    update_freq=1,
)

pandas.core.frame.DataFrame

In [11]:
dataloaders_dicts = DataLoaderKwargs(train_kwargs = {"batch_size":
                                                        config_data.batch_size_train,
                                                    "shuffle":True},
                                     val_kwargs = {"batch_size": 16},
                                     test_kwargs = {"batch_size": 3})

og = OrbitsGenerator(num_orbits_per_class=1_000,
                     homology_dimensions = config_data.homology_dimensions,
                     validation_percentage=0.2,
                     test_percentage=config_data.test_percentage,
                     n_jobs=config_data.num_jobs,
                     dynamical_system = config_data.dynamical_system,
                     dtype=config_data.dtype,
                     arbitrary_precision=False,
                     )
dl_train, dl_val, _ = og.get_dataloader_persistence_diagrams(dataloaders_dicts)

In [ ]:
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score

class SetTransformerOld(pl.LightningModule):
    """ Vanilla SetTransformer from
    https://github.com/juho-lee/set_transformer/blob/master/main_pointcloud.py
    """
    def __init__(
        self,
        dim_input=4,  # dimension of input data for each element in the set
        num_outputs=1,
        dim_output=5,  # number of classes
        num_inds=32,  # number of induced points, see  Set Transformer paper
        dim_hidden=128,
        num_heads="4",
        layer_norm="False",  # use layer norm
        dropout=0.0,
        num_layer_enc=2,
        num_layer_dec=2,
        activation="nn.Relu()",
        bias_attention=True,
        attention_type="induced_attention"
    ):
        super().__init__()
        if attention_type=="induced_attention":
            self.enc = nn.Sequential(
                ISAB(dim_input, dim_hidden, eval(num_heads), num_inds, ln=eval(layer_norm), bias_attention=bias_attention),
                *[ISAB(dim_hidden, dim_hidden, eval(num_heads), num_inds, ln=eval(layer_norm), bias_attention=bias_attention)
                  for _ in range(num_layer_enc-1)],
            )
        else:
            self.enc = nn.Sequential(
                SAB(dim_input, dim_hidden, eval(num_heads), ln=eval(layer_norm), bias_attention=bias_attention),
                *[SAB(dim_hidden, dim_hidden, eval(num_heads), ln=eval(layer_norm), bias_attention=bias_attention)
                  for _ in range(num_layer_enc-1)],
            )
        self.dec = nn.Sequential(
            nn.Dropout(dropout),
            PMA(dim_hidden, eval(num_heads), num_outputs, ln=eval(layer_norm), bias_attention=bias_attention),
            nn.Dropout(dropout),
            *[nn.Sequential(nn.Linear(dim_hidden, dim_hidden),
                            nn.ReLU(),
                            nn.Dropout(dropout)) for _ in range(num_layer_dec-1)],
            nn.Linear(dim_hidden, dim_output),
        )
        
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input):
        return self.dec(self.enc(input)).squeeze(dim=1)
    
    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        x_hat = self.forward(x)
        loss = self.loss_fn(x_hat, y)
        # Logging to TensorBoard by default
        self.log("train_loss", loss)
        print(loss)
        return loss
    
    def val_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        x_hat = self.forward(x)
        acc = accuracy_score(x_hat, y)
        # Logging to TensorBoard by default
        print("acc", acc)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


model = SetTransformerOld(dim_input=4, dim_output=5,
                       num_inds=config_model.num_induced_points,
                       dim_hidden=config_model.dim_hidden,
                       num_heads=str(config_model.num_heads),
                       layer_norm=str(config_model.layer_norm),  # use layer norm
                       dropout=config_model.dropout_dec,
                       num_layer_enc=config_model.num_layers_encoder,
                       num_layer_dec=config_model.num_layers_decoder,
                       activation=config_model.activation,
                       bias_attention=config_model.bias_attention,
                       attention_type=config_model.attention_type)

trainer = pl.Trainer(max_epochs=100, log_every_n_steps=1, gpus=1)
trainer.fit(model, dl_train)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | enc     | Sequential       | 249 K 
1 | dec     | Sequential       | 83.3 K
2 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
332 K     Trainable params
0         Non-trainable params
332 K     Total params
1.331     Total estimated model params size (MB)


tensor(1.5949, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6709, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6257, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6032, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6205, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6077, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5960, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6714, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6471, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6020, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6284, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6223, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6285, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6213, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6319, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6369, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6398, device='cuda:0', grad_fn=

tensor(1.6116, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6247, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6184, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6253, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6224, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6240, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5977, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6073, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6099, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5949, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6077, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6179, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5993, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6136, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6106, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6188, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6090, device='cuda:0', grad_fn=

tensor(1.6020, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6048, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6125, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6202, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6303, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6200, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6189, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5975, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6144, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6052, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6188, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6036, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6165, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6243, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6073, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6309, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6259, device='cuda:0', grad_fn=

tensor(1.6101, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6108, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6056, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6171, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5995, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6265, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6203, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6115, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6098, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6066, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6033, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6155, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6191, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6062, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6068, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6079, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6060, device='cuda:0', grad_fn=

...

tensor(1.6101, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6088, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6083, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6087, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6041, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6054, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6101, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6205, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6198, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6078, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6072, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6092, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6070, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6118, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6130, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6112, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6103, device='cuda:0', grad_fn=

tensor(1.6115, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6091, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6121, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6062, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6116, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6071, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6017, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6234, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6227, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6106, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6071, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6091, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6124, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6099, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6080, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6115, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6103, device='cuda:0', grad_fn=

tensor(1.6071, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6112, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6092, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6088, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6140, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6117, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6093, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6082, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6114, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6091, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6043, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6067, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6076, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6149, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6113, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6115, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6077, device='cuda:0', grad_fn=

tensor(1.6111, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6104, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6078, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6085, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6100, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6062, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6106, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6090, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6111, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6105, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6077, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6110, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6055, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6096, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6116, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6072, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6085, device='cuda:0', grad_fn=

tensor(1.6115, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6066, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6087, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6096, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6071, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6082, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6087, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6095, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6082, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6112, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6086, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6078, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6092, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6091, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6119, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6116, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6095, device='cuda:0', grad_fn=